описание коннекторов, компаний и протокола
https://2chargers.net/api/filters

еще какой-то запросик, про мощности зарядок
https://2chargers.net/api/filters/occupy

все машины с коннекторами и всякое такое
https://2chargers.net/api/cars

кампании с описанием, телефонами, приложениями
https://2chargers.net/api/companies

запрос станций
https://2chargers.net/api/addresses

с жсоном (там всякие фильтры по коннекторам, по компаниям + география), кажется, выдача ограничена 500 адресами
{"userCoordinates":[[2.183847973273408,2.433936707675457],[79.08407537418269,76.8307976424694]],"connectorIds":[],"availableIds":[0,1,2,3],"companyIds":[45,44,42,41,40,39,38,37,36,35,34,32,31,30,29,28,27,26,25,24,23,22,21,19,18,17,16,13,12,11,10,8,7,6,5,4,3,2,1],"connectors":[{"id":2,"type_id":0},{"id":37,"type_id":0},{"id":38,"type_id":0},{"id":3,"type_id":0},{"id":39,"type_id":0},{"id":41,"type_id":0},{"id":40,"type_id":0},{"id":1,"type_id":0},{"id":50,"type_id":0},{"id":7,"type_id":0},{"id":4,"type_id":0},{"id":17,"type_id":0},{"id":42,"type_id":0},{"id":5,"type_id":0},{"id":13,"type_id":0},{"id":12,"type_id":0},{"id":9,"type_id":0},{"id":23,"type_id":0},{"id":24,"type_id":0},{"id":25,"type_id":0},{"id":26,"type_id":0},{"id":14,"type_id":0},{"id":11,"type_id":0},{"id":10,"type_id":0},{"id":49,"type_id":0},{"id":30,"type_id":0},{"id":44,"type_id":0},{"id":8,"type_id":0},{"id":19,"type_id":0},{"id":29,"type_id":0},{"id":6,"type_id":0},{"id":33,"type_id":0},{"id":34,"type_id":0},{"id":31,"type_id":0},{"id":32,"type_id":0},{"id":35,"type_id":0},{"id":15,"type_id":0},{"id":43,"type_id":0},{"id":16,"type_id":0},{"id":18,"type_id":0},{"id":36,"type_id":0},{"id":28,"type_id":0},{"id":27,"type_id":0}],"stationTypeIds":[],"zoomLevel":3.0,"onlyFast":false,"car_id":null,"kilowattRange":{"min":0.0,"max":0.0},"showStationsWithoutKwt":false,"withoutCompany":false,"hideDeleted":false}

пример станции

по конкретной станции собирается так (post/get)
https://2chargers.net/api/addresses/8761/stations

запрос трафика по дню недели (get), в конце цифра - номер для недели
https://2chargers.net/api/addresses/8761/traffic/1

не знаю что это (get)
https://2chargers.net/api/addresses/8761/active-marks

общая инфа по адресу, без мощностей (get)
https://2chargers.net/api/addresses/8761


In [2]:
import pandas as pd
import requests
import joblib
import ast

In [156]:
class TwoChargers():
    def __init__(self, base_url = 'https://2chargers.net/api/'):
        self.base_url = base_url
        self.post_methods = ['filters', 'cars', 'companies', 'addresses']
        self.get_methods = ['stations', 'traffic', 'loc_info']
    
    def request(self, method, json = None, loc_id = None, week_day = None):
        orig_method = method
        if method in self.post_methods:
            request = requests.post
        elif method in self.get_methods:
            request = requests.get
            if method == 'loc_info':
                method = f'addresses/{loc_id}'
            elif method == 'traffic':
                method = f'addresses/{loc_id}/{method}/{week_day}'
            elif method == 'stations':
                method = f'addresses/{loc_id}/{method}'
            else:
                print('unknown get method')
                return None    
        else:
            print('unknown method')
            return None
        r = request(f'{self.base_url}{method}', json=json)
        if r.status_code != 200:
            print(f'{r.status_code} for {self.base_url}{method}')
            return None
        else:            
            response_dict = r.json()
            method = orig_method
            return self.proc(method, response_dict)            
    
    def proc(self, method, response_dict):
        if method == 'filters':
            connectors = response_dict['connectors']
            for connector in connectors:
                connector['decription'] = connector['extra']['explanation']
                del connector['extra']
            connectors_df = pd.DataFrame.from_dict(connectors)
            connectors_df.to_csv('connectors.csv', encoding='utf-8', index=False)
            return connectors_df
            
        elif method == 'companies':
            companies_df = pd.DataFrame.from_dict(response_dict)
            companies_df.to_csv('companies.csv', encoding='utf-8', index=False)
            return companies_df
        
        elif method == 'cars':
            cars_df = pd.DataFrame.from_dict(response_dict)
            cars_df.to_csv('cars.csv', encoding='utf-8', index=False)
            return cars_df

        elif method == 'addresses':
            if len(response_dict) == 200:
                print('decrease area, not all stations scraped')
            return pd.DataFrame.from_dict(response_dict)
        
        elif method == 'traffic':
            return response_dict

        elif method in self.get_methods:
            return pd.DataFrame.from_dict(response_dict)                        
            
    def scan_all_segments(self):
        segments = joblib.load('segments')

        def divide_segment(segment):
            lat1, lon1, lat2, lon2 = segment[0][0], segment[0][1], segment[1][0], segment[1][1]
            return [[[lat1, lon1], [lat2, lon1 + (lon2-lon1)/2]], [[lat1, lon1 + (lon2-lon1)/2], [lat2, lon2]]]

        stations = []
        segments_to_remove = []
        new_segments = []
        for segment in segments:
            json = {"userCoordinates":segment,
                "connectorIds":[],
                }
            segment_stations = self.request('addresses', json)
            if len(segment_stations) == 200:
                print('need seg division')
                segments_to_remove.append(segment)
                seg1, seg2 = divide_segment(segment)
                new_segments.extend([seg1, seg2])                
            else:
                stations.append(segment_stations)
        
        for segment in new_segments:
            json = {"userCoordinates":segment,
                "connectorIds":[],
                }
            segment_stations = self.request('addresses', json)
            if len(segment_stations) == 200:
                print('ALARM, need new seg division')
                segments_to_remove.append(segment)
                seg1, seg2 = divide_segment(segment)
                new_segments.extend([seg1, seg2])                
            else:
                stations.append(segment_stations)
        
        # надо обновить список сегментов
        if new_segments:
            for segment in segments:
                if segment not in segments_to_remove:
                    new_segments.append(segment)

            joblib.dump(new_segments, 'segments')
        
        all_stations = pd.concat(stations)
        all_stations.loc[all_stations.country_id == 1].to_csv('ru_station_locations.csv', encoding='utf-8', index=False)
        return all_stations

    def scan_all_locations(self):
        stations = []
        ru_station_locations = pd.read_csv('ru_station_locations.csv')
        for idx, row in ru_station_locations.iterrows():
            loc_id = row.id
            st_info = self.request('stations', loc_id = loc_id)
            st_info['location_id'] = loc_id
            stations.append(st_info)
        all_stations = pd.concat(stations)
        all_stations.to_csv('all_stations.csv', encoding='utf-8', index=False)
        return all_stations

    def get_traffic(self):
        stations = []
        ru_station_locations = pd.read_csv('ru_station_locations.csv')
        for idx, row in ru_station_locations.iterrows():
            loc_id = row.id
            station_load = {}
            for day in range(1,8):
                station_load[day] = {}
                st_traffic = self.request('traffic', loc_id = loc_id, week_day=day)
                if not st_traffic:
                    break
                for h in st_traffic:
                    if h['count'] != 0:
                        station_load[day][h['hour']] = h['count']
            stations.append({'location_id' : loc_id, 'traffic' : str(station_load)})
        all_stations_traffic = pd.DataFrame.from_dict(stations)
        all_stations_traffic.to_csv('all_stations_traffic.csv', encoding='utf-8', index=False)
        return all_stations_traffic

    def format_output(self):        
        ru_station_locations = pd.read_csv('ru_station_locations.csv')
        ru_station_locations.fillna('', inplace=True)
        stations_data = pd.read_csv('all_stations.csv')
        stations_data.fillna('', inplace=True)

        stations = []

        connectors = {'type_2' : [7, 50],
                    'type_1' : [4],
                    'gbt' : [12, 13],
                    'ccs' : [2,37,38],
                    'chademo' : [3]}

        for _, location in ru_station_locations.iterrows():
            station = stations_data.loc[stations_data.location_id == location.id]

            powers = {}
            price = {}
            for key in connectors.keys():
                powers[key] = station.loc[station.connector_id.isin(connectors[key])].kwt.tolist()

                price[key] = station.loc[station.connector_id.isin(connectors[key])].price_per_kwt.tolist()
                if '' in powers[key]:
                    powers[key].remove('')

                if '' in price[key]:
                    price[key].remove('')
                
            for key in list(price.keys()):
                if len(price[key]) == 0:
                    del price[key]
                else:
                    price[key] = sum(price[key]) / len(price[key])

            if location.serving_company:
                company = ast.literal_eval(location.serving_company)['name']
            else:
                company = ''

            if location.working_all_time:
                working_time = '0-24'
            else:
                working_time = location.working_hours
            
            st_info = {'loc_id' : location.id,
            'name' : location['name'],
            'addr' : location.address,
            'coords' : f'{location.latitude}, {location.longitude}',
            'manufacturer' : '',
            'company' : company,
            'price' : price,
            'working_time' : working_time,
            'phone' : location.phone,
            'desc' : location.description,
            'type' : location.icon_type,
            'country' : location.country_id,
            'area' : location.area_id,
            'city' : location.city_id,
            'type_2_num' : len(powers['type_2']),
            'type_2_power' : sum(powers['type_2']),
            'ccs_num' : len(powers['ccs']),
            'ccs_power' : sum(powers['ccs']),
            'chademo_num' : len(powers['chademo']),
            'chademo_power' : sum(powers['chademo']),
            'gbt_num' : len(powers['gbt']),
            'gbt_power' : sum(powers['gbt']),
            'type_1_num' : len(powers['type_1']),
            'type_1_power' : sum(powers['type_1']),
            }
            
            stations.append(st_info)
        
        cols = ['loc_id', 'name', 'addr', 'coords', 'manufacturer', 'type_2_num', 'type_2_power', 
        'ccs_num', 'ccs_power', 'chademo_num', 'chademo_power', 'gbt_num', 'gbt_power', 'type_1_num', 'type_1_power',
        'company', 'price', 'working_time', 'phone', 'desc', 'type', 'country', 'area', 'city']

        needed_format = pd.DataFrame(columns=cols)
        needed_format = pd.concat([needed_format, pd.DataFrame.from_dict(stations)])
        needed_format.to_csv('stations_formatted.csv', encoding='utf-8', index=False)
        return needed_format


In [157]:
two_chargers_parser = TwoChargers()
df = two_chargers_parser.format_output()

In [85]:
two_chargers_parser = TwoChargers()
df = two_chargers_parser.get_traffic()

400 for https://2chargers.net/api/addresses/623/traffic/1
400 for https://2chargers.net/api/addresses/1041/traffic/1
400 for https://2chargers.net/api/addresses/5229/traffic/1
400 for https://2chargers.net/api/addresses/1042/traffic/1
400 for https://2chargers.net/api/addresses/927/traffic/1
400 for https://2chargers.net/api/addresses/501/traffic/1
400 for https://2chargers.net/api/addresses/655/traffic/1
400 for https://2chargers.net/api/addresses/38359/traffic/1
400 for https://2chargers.net/api/addresses/38332/traffic/1
400 for https://2chargers.net/api/addresses/982/traffic/1
400 for https://2chargers.net/api/addresses/1049/traffic/1
400 for https://2chargers.net/api/addresses/900/traffic/1
400 for https://2chargers.net/api/addresses/3193/traffic/1
400 for https://2chargers.net/api/addresses/2558/traffic/1
400 for https://2chargers.net/api/addresses/3388/traffic/1
400 for https://2chargers.net/api/addresses/37368/traffic/1
400 for https://2chargers.net/api/addresses/3418/traffic/1


In [47]:
two_chargers_parser = TwoChargers()
df = two_chargers_parser.scan_all_locations()

In [188]:
two_chargers_parser = TwoChargers()
df = two_chargers_parser.scan_all_segments()

In [ ]:
# делилка на сегменты, где возврат меньше 200 станций

In [155]:
needed_format

,loc_id,name,addr,coords,manufacturer,type_2_num,type_2_power,ccs_num,ccs_power,chademo_num,...,type_1_power,company,price,working_time,phone,desc,type,country,area,city
0,1675,,"ул. Пыталовские Хутора, Пыталово, Псковская об...","57.052302, 27.905914",,0,0.0,0,0.0,0,...,0.0,,{},,,,home_station,1.0,195.0,341.0
1,9935,"#22, #23, пос.Кондратьево, АЗС Татнефть",,"60.621583, 28.163497",,1,22.0,1,20.0,1,...,0.0,RosSeti,{},,,"#22, #23, пос.Кондратьево, АЗС Татнефть",public_station_fast,1.0,35.0,771.0
2,666,Ranch,"Печорский район, Псковская область, Россия","57.693853, 27.86853",,0,0.0,0,0.0,0,...,0.0,,{},0-24,+79113555555,Туристическая база Ranch. 220 розетка по догов...,public_station,1.0,195.0,
3,37300,Tesla,"Россия, Псковский район, деревня Загорицы, Око...","57.86031483014521, 28.195321820676327",,0,0.0,0,0.0,0,...,0.0,,{'type_2': 5.0},,+79113527208,Gen 3 Wall Connector от Тесла.\nЕсли вам ОЧЕНЬ...,public_station_paid,1.0,195.0,1649.0
4,1842,Несте,"Кингисеппское ш., 11, Ивангород, Ленинградская...","59.376083, 28.224545",,0,0.0,0,0.0,0,...,0.0,,{},0-24,,Розетка в шатре у бензоколонки Несте,public_station,1.0,35.0,531.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4705,5641,Спортивная Дом 8,"ул. Спортивная, Петропавловск-Камчатский, Камч...","52.993693, 158.770238",,0,0.0,0,0.0,0,...,0.0,,{},,,,public_station,1.0,248.0,947.0
4706,5629,EV Charger Station Супермаркет Гвоздь,Магистральная 50Б,"53.168491, 158.413877",,0,0.0,0,0.0,0,...,0.0,,{},,,,public_station_fast,1.0,248.0,419.0
4707,5632,EV Charger Station Новое здание на 10км,Проспект победы д.20/1,"53.071638, 158.595996",,0,0.0,1,50.0,1,...,0.0,,{},0-24,,CCS Combo - 50кВт\nChademo- 50кВт\nMulti - 22к...,public_station_fast,1.0,248.0,947.0
4708,5633,EV Charger Station TЦ Фамилион,Северо-Восточное шоссе 6,"53.066008, 158.642109",,0,0.0,1,24.0,1,...,0.0,,{},0-24,,CCS Combo - 24кВт\nChademo- 24кВт\n\nЗапуск за...,public_station_fast,1.0,248.0,947.0


In [104]:
# формат гео - lat, lon - lat, lon, например - [[55.765158, 49.18338],[55.965158, 49.58338]]
#'latitude': 55.765158,
#'longitude': 49.18338

#оригинальный запрос
json = {"userCoordinates":[[55.765158, 49.18338],[55.965158, 49.58338]],
"connectorIds":[],
"availableIds":[0,1,2,3],
"companyIds":[45,44,42,41,40,39,38,37,36,35,34,32,31,30,29,28,27,26,25,24,23,22,21,
                19,18,17,16,13,12,11,10,8,7,6,5,4,3,2,1],
"connectors":[{"id":2,"type_id":0},{"id":37,"type_id":0},{"id":38,"type_id":0},
                {"id":3,"type_id":0},{"id":39,"type_id":0},{"id":41,"type_id":0},
                {"id":40,"type_id":0},{"id":1,"type_id":0},{"id":50,"type_id":0},
                {"id":7,"type_id":0},{"id":4,"type_id":0},{"id":17,"type_id":0},
                {"id":42,"type_id":0},{"id":5,"type_id":0},{"id":13,"type_id":0},
                {"id":12,"type_id":0},{"id":9,"type_id":0},{"id":23,"type_id":0},
                {"id":24,"type_id":0},{"id":25,"type_id":0},{"id":26,"type_id":0},
                {"id":14,"type_id":0},{"id":11,"type_id":0},{"id":10,"type_id":0},
                {"id":49,"type_id":0},{"id":30,"type_id":0},{"id":44,"type_id":0},
                {"id":8,"type_id":0},{"id":19,"type_id":0},{"id":29,"type_id":0},
                {"id":6,"type_id":0},{"id":33,"type_id":0},{"id":34,"type_id":0},
                {"id":31,"type_id":0},{"id":32,"type_id":0},{"id":35,"type_id":0},
                {"id":15,"type_id":0},{"id":43,"type_id":0},{"id":16,"type_id":0},
                {"id":18,"type_id":0},{"id":36,"type_id":0},{"id":28,"type_id":0},
                {"id":27,"type_id":0}],
"stationTypeIds":[],
"zoomLevel":9.0,
"onlyFast":False,
"car_id":None,
"kilowattRange":{"min":0.0,"max":0.0},
"showStationsWithoutKwt":False,
"withoutCompany":False,
"hideDeleted":False}

# минимальный
json = {"userCoordinates":[[55.765158, 49.18338],[55.965158, 49.58338]],
"connectorIds":[],
}

с нуля заполняю сегменты

In [164]:
# есть какая-то проблема с переходом от -180 + 180, поэтому делим сразу на сегмента без этого перехода
initial_segments = [[[38.71443081148476, 27.055894707637356], [78.83427199842515, 180]], [[38.71443081148476, -180], [78.83427199842515, -168.4183446606007]]]
new_segments = []

def divide_segment(segment):
    lat1, lon1, lat2, lon2 = segment[0][0], segment[0][1], segment[1][0], segment[1][1]
    return [[[lat1, lon1], [lat2, lon1 + (lon2-lon1)/2]], [[lat1, lon1 + (lon2-lon1)/2], [lat2, lon2]]]

def get_loc(segment):
    global new_segments
    print(f'cur seg - {segment}')
    json = {"userCoordinates":segment,
    "connectorIds":[],
    }
    r = requests.post('https://2chargers.net/api/addresses', json=json)
    if r.status_code != 200:
        print(r)
        print(f'error - {r.status_code} on seg {segment}')
        #break
    else:
        response_dict = r.json()
        if len(response_dict) == 200:
            print(f'seg division - {segment}')
            seg1, seg2 = divide_segment(segment)
            print(f'new segs - {seg1}, {seg2}')
            get_loc(seg1)
            get_loc(seg2)            
        else:
            print(f'seg append, number of stations - {len(response_dict)} - {segment}')
            new_segments.append(segment)
            return None

        #companies_df = pd.DataFrame.from_dict(response_dict['companies'])
        #companies_df.to_csv('companies.csv', encoding='utf-8', index=False)

for seg in initial_segments:
    get_loc(seg)

cur seg - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 180]]
seg division - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 180]]
new segs - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 103.52794735381869]], [[38.71443081148476, 103.52794735381869], [78.83427199842515, 180]]
cur seg - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 103.52794735381869]]
seg division - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 103.52794735381869]]
new segs - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 65.29192103072802]], [[38.71443081148476, 65.29192103072802], [78.83427199842515, 103.52794735381869]]
cur seg - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 65.29192103072802]]
seg division - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 65.29192103072802]]
new segs - [[38.71443081148476, 27.055894707637357], [78.83427199842515, 46.173907869182685]], [[38.71443081148476, 46.1

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a0668b3-9bed-4dec-8f71-91785d133502' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>